# Classes and Inheritance

As an OOP language, Python supports a full range of features, such as inheritance, polymorphism, and encapsulation.

Python's classes and inheritance make it easy to express your program's intended hevaviors with objects. They allow you to improve and expand functionality over time. They provide flexibility in an env of changing requirements. **Knowing how to use them well enables u to write maintainable code**.


## Item 22: Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples

Python’s built-in dictionary type is wonderful for maintaining dynamic internal state over the lifetime of an object. By **dynamic**, I mean situations in which you need to do bookkeeping for an unexpected set of identifiers. For example, say you want to record the grades of a set of students whose names aren’t known in advance.

```python
class SimpleGradebook(object):
	def __init__(self):
		self._grades = {}

	def add_student(self, name):
		self._grades[name] = []

	def report_grade(self, name, score):
		self._grades[name].append(score)

	def average_grade(self, name):
		grades = self._grades[name]
		return sum(grades) / len(grades)
		

book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isasc Newton', 90)
print(book.average_grade('Isaac Newton'))
```

If you want to track scores by subject, you can use nested dict for it. Then if you want to track weights of scores also, you can use a tuple (score, weight) instead of a single score, here your code would look like this:

```python
book.report_grade('Albert Einstein', 'Math', 80, 0.10)
```

When you see complexity like this happen, it's time to make the leap from dictionaries and tuples to a hierarchy of classes.

At first, you didn’t know you’d need to support weighted grades, so the complexity of additional helper classes seemed unwarranted. Python’s built-in dictionary and tuple types made it easy to keep going, adding layer after layer to the internal bookkeeping. But you should avoid doing this for more than one level of nesting (i.e., **avoid dictionaries that contain dictionaries**). It makes your code hard to read by other programmers and sets you up for a maintenance nightmare.

**As soon as you realize the bookkeeping is getting complicated, break it all out into classes**. This lets you provide **well-defined interfaces that better encapsulate your data**. This also enables you to **create a layer of abstraction between your interfaces and your concrete implementations**.

Also, a pattern of extending tuples longer and longer is similar to deepening layers of dictionaries. As soon as you find yourself **going longer than a two-tuple, it’s time to consider another approach**.

Consider using `namedtuple`:

```python
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))
```

Having named attributes makes it easy to move from a namedtuple to your own class later if your requirements change again and you need to add behaviors to the simple data containers.

Other codes:

```python
class Subject(object):
	def __init__(self):
		self._grades = []

	def report_grade(self, score, weight):
		self._grades.append(Grade(score, weight))

	def average_grade(self, name):
		total, total_weight = 0, 0
		for grade in self._grades:
			total += grade.score * grade.weight
			total_weight += grade.weight
		return total / total_weight


class Student(object):
	def __init__(self):
		self._subjects = {}

	def subject(self, name):
		if name not in self._subjects:
			self._subjects[name] = Subject()
		return self._subjects[name]

	def average_grade(self):
		total, count = 0, 0
		for subject in self._subjects.values():
			total += subject.average_grade()
			count += 1
		return total / count

class Gradebook(object):
	def __init__(self):
		self._students = {}

	def student(self, name):
		if name not in self._students:
			self._students[name] = Student()
		return self._students[name]


book = SimpleGradebook()
albert = book.student('Albert Einstein')
math = albert.subject('Math')
math.report_grade(80, 0.10)
#...
print(albert.average_grade())
```

## Item 23: Accept Functions for Simple Interfaces Instead of Classes

Many of Python’s built-in APIs allow you to customize behavior by passing in a function. These hooks are used by APIs to call back your code while they execute. For example, the `list` type’s `sort` method takes an optional `key` argument that’s used to determine each index’s value for sorting.

```python
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=lambda x: len(x))
print(names)
```

In other languages, you might expect hooks to be defined by an abstract class. (hey, Java) In Python, many hooks are just stateless functions with well-defined arguments and return values. **Functions are ideal for hooks because they are easier to describe and simpler to define than classes.** Functions work as hooks because Python has first-class functions: Functions and methods can be passed around and referenced like any other value in the language.

For using defaultdict with a specific missing func, we can do like this:

```python
from collections import defaultdict


class CountMissing(object):
    def __init__(self):
        self.added = 0

    def missing(self):
        self.added += 1
        return 0


current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

counter = CountMissing()
result = defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount

assert counter.added == 2
```

It seems nice, but it's not immediately obvious what the purpose of the `CountMissing` class is. To clarify this situation, Python allows classes to define the `__call__` special method. `__call__` allows an object to be called just like a function. It also affects the `callable` func.

```python
from collections import defaultdict


class BetterCountMissing(object):
    def __init__(self):
        self.added = 0

    def __call__(self):
        self.added += 1
        return 0


current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

counter = BetterCountMissing()
result = defaultdict(counter, current)

for key, amount in increments:
    result[key] += amount

assert counter.added == 2
```

It provides a strong hint that **the goal of the class is to act as a stateful closure**.

## Item 24: Use @classmethod Polymorphism to Construct Objects Generically

In Python, not only do the objects support polymorphism (duck type?), but the classes do as well (inheritance?).

**Polymorphism** is a way for multiple classes in a hierarchy to implement their own unique versions of a method. This allows many classes to fulfill the same interface or abstract base class while providing different functionality.

Define an **abstract** method:

```python
class InputData(object):
    def read(self):
        raise NotImplementedError


class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        return open(self.path).read()
```

Say we also define a `Worker` abstract class and a `LineCountWorker` subclass which are implemeted by InputData interface.

Here comes with the question: what connects all of these pieces? i.e. what's responsible for building the objects?

This problem boils down to needing a generic way to construct objects. 

In other languages, you’d solve this problem with **constructor polymorphism**, requiring that each InputData subclass provides a special constructor that can be used generically by the helper methods that orchestrate the MapReduce. The trouble is that Python only allows for the single constructor method **__init__**.

The best way to solve this is with `@classmethod` polymorphism. 

```python
import os

class GenericInputData(object):
    def read(self):
        raise NotImplementedError

    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError


class PathInputData(GenericInputData):
    
    def __init__(self, path):
        super().__init__()
        self.path = path
    
    def read(self):
        return open(self.path).read()

    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))
```

Things to Remember
* Python only supports a single constructor per class, the __init__ method.
* Use @classmethod to define alternative constructors for your classes.
* Use class method polymorphism to provide generic ways to build and connect concrete subclasses.


## Item 25: Initialize Parent Classes with super

The old way to initialize a parent class from a child class is to directly call the parent class’s `__init__` method with the child instance.

This approach works fine for simple hierarchies but breaks down in many cases. 
If your class is affected by **multiple inheritance** (something to avoid in general), calling the superclasses' `__init__` methods directly can lead to unpredictable behavior.

* the `__init__` call order isn't specified across all subclasses.
* diamond inheritance.

To solve these problems, Python 2.2 added the `super` built-in func and defined the method resolution order(MRO).

PS: Python 2 uses a verbose version of `super`.

Python 3:

```python
class Explicit(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value * 2)

class Implicit(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 2)

assert Explicit(10).value == Implicit(10).value
```

Things to Remember

* Python’s standard method resolution order (MRO) solves the problems of superclass initialization order and diamond inheritance.
* Always use the super built-in function to initialize parent classes.

## Item 26: Use Multiple Inheritance Only for Mix-in Utility Classes

Python is an object-oriented language with built-in facilities for making multiple inheritance tractable. However, **it’s better to avoid multiple inheritance altogether**.

If you find yourself desiring the convenience and encapsulation that comes with multiple inheritance, consider writing a **mix-in** instead. A mix-in is a small class that only defines a set of additional methods that a class should provide. Mix-in classes don’t define their own instance attributes nor require their `__init__` constructor to be called.

Writing mix-ins is easy because Python makes it trivial to inspect the current state of any object regardless of its type. Dynamic inspection lets you write generic functionality a single time, in a mix-in, that **can be applied to many other classes**. Mix-ins can be composed and layered to minimize repetitive code and maximize reuse.

For example, say you want the ability to convert a Python object from its in-memory representation to a dictionary that’s ready for serialization. **Why not write this functionality generically so you can use it with all of your classes?**

Here, I def an example mix-in that accomplishes this with a new public method that's added to any class that inheirts from it:

```python
class ToDictMixin(object):
    def to_dict(self):
        return self._traverse_dict(self.__dict__)

    def _traverse_dict(self, instance_dict):

        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output


    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value
```

Use the mix-in to make a dict repr of a binary tree:

```python
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right


tree = BinaryTree(10,
                  left=BinaryTree(7, right=BinaryTree(9)),
                  right=BinaryTree(13, left=BinaryTree(11)))
print(tree.to_dict())
>>>
{'right': {'right': None, 'value': 13, 'left': {'right': None, 'value': 11, 'left': None}}, 'value': 10, 'left': {'right': {'right': None, 'value': 9, 'left': None}, 'value': 7, 'left': None}}
```

The best part about mix-in is that **you can make their generic functionality pluggable so behaviors can be overridden when required**.

Things to Remember
* Avoid using multiple inheritance if mix-in classes can achieve the same outcome.
* Use pluggable behaviors at the instance level to provide per-class customization when mix-in classes may require it.
* Compose mix-ins to create complex functionality from simple behaviors.

PS: Mix-in classes look like a kind of **thin class definition**.

## Item 27: Perfer Public Attributes Over Private Ones

In Python, there are only two types of attribute visibility for a class' attributes: *public* and *private*.

```python
class MyObject(object):
	def __init__(self):
		self.public_field = 5
		self.__private_field = 10

	def get_private_field(self):
		return self.__private_field

	@classmethod
	def get_private_field_of_instance(cls, instance):
		return instance.__private_field


foo = MyObject()
assert foo.public_field == 5

# access private field directly
# AttributeError
#print(foo.__private_field)

# by class method
bar = MyObject()
assert MyObject.get_private_field_of_instance(bar)
```

For private fields, **a subclass can't access its parent class's private fields**. (so **protected visibility is not available**)

```python
class MyParentObj(object):
	def __init__(self):
		self.__private_field = 71


class MyChildObj(MyParentObj):
	def get_private_field(self):
		return self.__private_field
		

baz = MyChildObj()
# AttributeError
print(baz.get_private_field())
```

The private attribute behavior is implemented with a simple transformation of the attribute name. When the Python compiler sees private attribute access in methods like `MyChildObject.get_private_field`, it translates `__private_field` to access `_MyChildObject__private_field` instead.

Knowing this scheme, you can easily access the private attributes of any class, **from a subclass or externally, without asking for permission**.

```python
assert baz._MyParentObj__private_field == 71

print(baz.__dict__)
#{'_MyParentObj__private_field': 71}
```

Why doesn’t the syntax for private attributes actually enforce strict visibility? The simplest answer is one often-quoted motto of Python: **“We are all consenting adults here.”** Python programmers believe that the benefits of being open outweigh the downsides of being closed.

Beyond that, having the ability to hook language features like attribute access (see Item 32: “Use `__getattr__`, `__getattribute__`, and `__setattr__` for Lazy Attributes”) enables you to mess around with the internals of objects whenever you wish.

To minimize the damage of accessing internals unknowingly, Python programmers follow a naming convention defined in the style guide (PEP 8). Fields prefixed by a single underscore (`_protected_field`) are *protected*.

Things to Remember

* Private attributes aren’t rigorously enforced by the Python compiler.
* Plan from the beginning to allow subclasses to do more with your internal APIs and attributes instead of locking them out by default.
* Use documentation of protected fields to guide subclasses instead of trying to force access control with private attributes.
* Only consider using private attributes to avoid naming conflicts with subclasses that are out of your control.

## Item 28: Inherit from collections.abc for Custom Container Types

Python provides built-in container types for managing data: list, tuples, sets, and dicts.

When you're designing classes for simple use cases like seqences, it's natural that you'd want to subclass Python's built-in `list` type directly.

```python
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)

    def frequency(self):
        counts = {}
        for item in self:
            counts.setdefault(item, 0)
            counts[item] += 1
        return counts


if __name__ == '__main__':
    foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
    print('Length is ', len(foo))
    foo.pop()
    print('After pop: ', repr(foo))
    print('Frequency: ', foo.frequency())
    
>>>
Length is  7
After pop:  ['a', 'b', 'a', 'c', 'b', 'a']
Frequency:  {'b': 2, 'a': 3, 'c': 1}
```

Imagine you want to provide an object that feels like a list, allowing indexing, but isn't a list subclass. For indexing, you can add `__getitem__` method, for length, you can add `__len__` method. But that's not enough, `count` and `index` methods are also expected to be contained in a seq obj like list or tuple.

To avoid this difficulty throughout the Python universe, the built-in `collections.abc` module defines a set of abstract base classes that provide all of the typical methods for each container type. When you subclass from these abstract base classes and forget to implement required methods, the module will tell you sth is wrong.

```python
from collections.abc import Sequence

class BadType(Sequence):
    pass

foo = BadType()
# TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__
```

Things to Remember
* Inherit directly from Python’s container types (like list or dict) for simple use cases.
* Beware of the large number of methods required to implement custom container types correctly.
* Have your custom container types inherit from the interfaces defined in collections.abc to ensure that your classes match required interfaces and behaviors.